In [3]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
# URL del archivo en el bucket público de AWS
url = "https://hybridge-education-machine-learning-datasets.s3.us-east-1.amazonaws.com/Fraud.csv"
df = pd.read_csv(url)

In [6]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [8]:
len(df)

6362620

In [9]:
# Inspeccionar el tamaño y las columnas
print("--- Información General del DataFrame ---")
df.info()

# Contar valores nulos (NaN)
print("\n--- Conteo de Valores Nulos ---")
print(df.isnull().sum())

# Mostrar estadísticas descriptivas
print("\n--- Estadísticas Descriptivas de Variables Numéricas ---")
print(df.describe())

--- Información General del DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB

--- Conteo de Valores Nulos ---
step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

--- Estadísticas Descriptivas de Variables Numéricas ---
               step        amount  oldbalanceOrg  newbalanceOrig  \
count  6.362

In [10]:
# Distribución de la variable objetivo
print("\n--- Distribución de la Clase 'isFraud' ---")
print(df['isFraud'].value_counts())

# Calcular el porcentaje de transacciones fraudulentas
fraud_percentage = df['isFraud'].sum() / len(df) * 100
print(f"\nPorcentaje de transacciones fraudulentas: {fraud_percentage:.4f}%")


--- Distribución de la Clase 'isFraud' ---
isFraud
0    6354407
1       8213
Name: count, dtype: int64

Porcentaje de transacciones fraudulentas: 0.1291%


In [11]:
# 1. Eliminar columnas no informativas o con alta cardinalidad
df = df.drop(['step', 'nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)

# 2. Convertir la columna 'type' usando One-Hot Encoding
df = pd.get_dummies(df, columns=['type'], drop_first=True)

print("\n--- DataFrame después de la Codificación One-Hot ---")
print(df.head())


--- DataFrame después de la Codificación One-Hot ---
     amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  newbalanceDest  \
0   9839.64       170136.0       160296.36             0.0             0.0   
1   1864.28        21249.0        19384.72             0.0             0.0   
2    181.00          181.0            0.00             0.0             0.0   
3    181.00          181.0            0.00         21182.0             0.0   
4  11668.14        41554.0        29885.86             0.0             0.0   

   isFraud  type_CASH_OUT  type_DEBIT  type_PAYMENT  type_TRANSFER  
0        0          False       False          True          False  
1        0          False       False          True          False  
2        1          False       False         False           True  
3        1           True       False         False          False  
4        0          False       False          True          False  


In [12]:
# Definir las características (X) y la variable objetivo (y)
X = df.drop('isFraud', axis=1)
y = df['isFraud']

print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

Forma de X: (6362620, 9)
Forma de y: (6362620,)


In [13]:
# Dividir el conjunto de datos en entrenamiento (70%) y prueba (30%)
# El parámetro 'stratify=y' ayuda a mantener la misma proporción de clases en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("\n--- División de Datos ---")
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")


--- División de Datos ---
Tamaño de X_train: (4453834, 9)
Tamaño de X_test: (1908786, 9)


In [14]:
# Inicializar el escalador
scaler = StandardScaler()

# Ajustar el escalador SÓLO con los datos de entrenamiento y transformar ambos conjuntos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir de nuevo a DataFrame (opcional, pero útil para inspección)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\n--- Datos de Entrenamiento Escalados (Primeras Filas) ---")
print(X_train_scaled.head())


--- Datos de Entrenamiento Escalados (Primeras Filas) ---
     amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  newbalanceDest  \
0  0.133805      -0.277631       -0.192460        1.042527        0.859216   
1  2.005310      -0.139123       -0.292551       -0.324812       -0.052308   
2  1.595985      -0.268089       -0.292551       -0.320385       -0.018948   
3 -0.017525      -0.261134       -0.207299        0.108104        0.019806   
4 -0.290836       1.783478        1.756078        0.150109        0.103563   

   type_CASH_OUT  type_DEBIT  type_PAYMENT  type_TRANSFER  
0      -0.736366   -0.080899     -0.714817      -0.302349  
1      -0.736366   -0.080899     -0.714817       3.307441  
2      -0.736366   -0.080899     -0.714817       3.307441  
3      -0.736366   -0.080899     -0.714817      -0.302349  
4      -0.736366   -0.080899     -0.714817      -0.302349  


In [15]:
# Inicializar el modelo de Regresión Logística
# El parámetro 'class_weight='balanced'' es muy útil aquí por el desbalance de clases
log_reg_model = LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')

# Entrenar el modelo
log_reg_model.fit(X_train_scaled, y_train)

print("\n--- Entrenamiento Completado ---")


--- Entrenamiento Completado ---


In [ ]:
# Realizar predicciones sobre el conjunto de prueba
y_pred = log_reg_model.predict(X_test_scaled)

# 1. Matriz de Confusión
print("\n--- Matriz de Confusión ---")
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
# 

# 2. Reporte de Clasificación (Precisión, Recall, F1-Score)
print("\n--- Reporte de Clasificación ---")
print(classification_report(y_test, y_pred))